In [1]:
import pandas as pd
pd.set_option('display.max_columns', None)
import json
import numpy as np
from scipy.sparse import save_npz, csc_matrix
from sklearn.preprocessing import OneHotEncoder
#from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import MinMaxScaler
from tqdm import tqdm
tqdm.pandas()

### Смотрим на данные

In [2]:
data1 = pd.read_csv('lst_announcement_data_1.gzip', compression='gzip')
print('1')
data2 = pd.read_csv('lst_announcement_data_2.gzip', compression='gzip')
print('2')
data3 = pd.read_csv('lst_announcement_data_3.gzip', compression='gzip')
print('3')

1
2
3


In [3]:
print(data1.shape)
data1.head()

(1000000, 35)


,id,floornumber,category,bargainterms,description,totalarea,geo,building,land,hasfurniture,roomscount,hasconditioner,haskitchenfurniture,hastv,haswasher,hasbathtub,hasgarage,hasshower,bedroomscount,hasdishwasher,repairtype,petsallowed,hasbathhouse,hasfridge,wclocationtype,kitchenarea,roomarea,isapartments,loggiascount,balconiescount,allroomsarea,livingarea,flattype,combinedwcscount,ptn_dadd
0,191401467,NaN,townhouseSale,"{""price"":5.5E7,""currency"":""rur"",""priceType"":""a...",Все предложения в Барвихе! ОКП Барвиха Клаб. В...,400.00,"{""countryId"":138,""undergrounds"":[],""calculated...","{""materialType"":""monolith"",""parking"":{},""floor...","{""status"":""individualHousingConstruction"",""are...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,NaN,5.0,NaN,NaN,NaN,True,NaN,indoors,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022-08-01
1,212158376,NaN,commercialLandRent,"{""clientFee"":0,""leaseTermType"":""longTerm"",""pri...",Сдается от собственника в аренду площадка 560 ...,NaN,"{""countryId"":138,""undergrounds"":[],""calculated...","{""parking"":{},""cranageTypes"":[],""extinguishing...","{""status"":""settlements"",""possibleToChangeStatu...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022-08-01
2,227815832,NaN,landSale,"{""price"":240000.0,""currency"":""rur"",""priceType""...",Продам участок в жилом районе с. ХОМУТОВО,NaN,"{""countryId"":138,""undergrounds"":[],""calculated...","{""parking"":{},""cranageTypes"":[],""extinguishing...","{""status"":""individualHousingConstruction"",""are...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022-08-01
3,238017518,1.0,flatRent,"{""clientFee"":0,""leaseTermType"":""longTerm"",""pri...","Привет, наконец ты попал сюда!\n\nУютная студи...",27.00,"{""countryId"":138,""undergrounds"":[],""calculated...","{""materialType"":""brick"",""parking"":{},""floorsCo...",{},True,NaN,True,True,NaN,True,True,NaN,NaN,NaN,NaN,euro,False,NaN,True,NaN,NaN,NaN,False,NaN,1.0,NaN,NaN,studio,NaN,2022-08-01
4,241597454,20.0,flatSale,"{""price"":1.6E7,""currency"":""rur"",""priceType"":""a...",Видовой 20 этаж. Корпус Д. Срочная перепродажа...,60.65,"{""countryId"":138,""undergrounds"":[],""calculated...","{""name"":""ЖК «CASABLANKA (Касабланка)»"",""materi...",{},NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,no,NaN,NaN,NaN,NaN,22.0,NaN,False,1.0,1.0,18+20,NaN,rooms,NaN,2022-08-01


In [4]:
print(data2.shape)
data2.head()

(1000000, 35)


,id,floornumber,category,bargainterms,description,totalarea,geo,building,land,hasfurniture,roomscount,hasconditioner,haskitchenfurniture,hastv,haswasher,hasbathtub,hasgarage,hasshower,bedroomscount,hasdishwasher,repairtype,petsallowed,hasbathhouse,hasfridge,wclocationtype,kitchenarea,roomarea,isapartments,loggiascount,balconiescount,allroomsarea,livingarea,flattype,combinedwcscount,ptn_dadd
0,275557007,13.0,flatSale,"{""price"":4200000.0,""currency"":""rur"",""priceType...",Продаю свою квартиру(Студию). Новый ремонт. Ку...,24.0,"{""countryId"":138,""undergrounds"":[],""calculated...","{""cargoLiftsCount"":2,""name"":""ЖК «Цветы»"",""mate...",{},NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,euro,NaN,NaN,NaN,NaN,NaN,NaN,False,NaN,NaN,NaN,NaN,studio,1.0,2022-08-01
1,275619810,4.0,flatSale,"{""price"":7450000.0,""currency"":""rur"",""priceType...",Представляем вашему вниманию отличную трехкомн...,74.5,"{""countryId"":138,""undergrounds"":[],""calculated...","{""cargoLiftsCount"":0,""parking"":{},""passengerLi...",{},NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,euro,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,1.0,NaN,NaN,rooms,0.0,2022-08-01
2,275743906,2.0,flatRent,"{""clientFee"":50,""leaseTermType"":""longTerm"",""pr...",Сдается однокомнатная квартира! Улучшенная пла...,40.0,"{""countryId"":138,""undergrounds"":[],""calculated...","{""materialType"":""brick"",""hasGarbageChute"":true...",{},True,1.0,NaN,True,True,True,True,NaN,NaN,NaN,NaN,cosmetic,True,NaN,True,NaN,9.0,NaN,False,1.0,NaN,NaN,20.0,rooms,NaN,2022-08-01
3,275961415,8.0,flatSale,"{""price"":6482000.0,""currency"":""rur"",""priceType...","Кирпичный дом, толщина кирпича 60 см. Индивиду...",86.2,"{""countryId"":138,""undergrounds"":[],""calculated...","{""name"":""ЖК «ул. Гражданская, 6»"",""parking"":{}...",{},NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,no,NaN,NaN,NaN,NaN,NaN,NaN,False,NaN,1.0,NaN,NaN,rooms,NaN,2022-08-01
4,275966988,2.0,flatSale,"{""price"":3200000.0,""currency"":""rur"",""priceType...",Внимание!\nВ продаже однокомнатная квартира в ...,31.0,"{""countryId"":138,""undergrounds"":[],""calculated...","{""materialType"":""brick"",""parking"":{},""floorsCo...",{},NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,no,NaN,NaN,NaN,NaN,6.0,NaN,NaN,0.0,0.0,NaN,18.0,rooms,1.0,2022-08-01


In [5]:
print(data3.shape)
data3.head()

(778005, 35)


,id,floornumber,category,bargainterms,description,totalarea,geo,building,land,hasfurniture,roomscount,hasconditioner,haskitchenfurniture,hastv,haswasher,hasbathtub,hasgarage,hasshower,bedroomscount,hasdishwasher,repairtype,petsallowed,hasbathhouse,hasfridge,wclocationtype,kitchenarea,roomarea,isapartments,loggiascount,balconiescount,allroomsarea,livingarea,flattype,combinedwcscount,ptn_dadd
0,274482535,NaN,garageSale,"{""price"":2500000.0,""currency"":""rur"",""contractT...",Продам сухой тёплый гаражный бокс с вентиляцие...,40.0,"{""countryId"":138,""undergrounds"":[],""calculated...","{""parking"":{""type"":""underground""},""cranageType...",{},NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022-08-01
1,274496484,2.0,flatSale,"{""price"":5800000.0,""currency"":""rur"",""priceType...",В продаже 2-комнатная квартира в тихом и споко...,40.8,"{""countryId"":138,""undergrounds"":[],""calculated...","{""materialType"":""brick"",""parking"":{},""floorsCo...",{},NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,no,NaN,NaN,NaN,NaN,5.0,NaN,False,NaN,1.0,17.8-9.2,27.0,rooms,NaN,2022-08-01
2,274683091,NaN,houseSale,"{""price"":3.5E7,""currency"":""rur"",""priceType"":""a...","1684491 ""Дом твоей мечты"" в пос. Ушково, г. Зе...",247.2,"{""countryId"":138,""undergrounds"":[{""transportTy...","{""parking"":{},""floorsCount"":2,""cranageTypes"":[...","{""status"":""individualHousingConstruction"",""are...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,NaN,NaN,NaN,cosmetic,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022-08-01
3,274794814,5.0,flatRent,"{""clientFee"":70,""leaseTermType"":""longTerm"",""pr...",Сдается однокомнатная квартира в Жк Весна\nВ к...,42.0,"{""countryId"":138,""undergrounds"":[],""calculated...","{""cargoLiftsCount"":0,""hasGarbageChute"":false,""...",{},True,1.0,True,True,True,True,True,NaN,False,NaN,False,cosmetic,False,NaN,True,NaN,NaN,NaN,NaN,0.0,1.0,NaN,NaN,rooms,1.0,2022-08-01
4,275064565,NaN,houseSale,"{""price"":3500000.0,""currency"":""rur"",""priceType...",Арт. 26169818 \nПродаю дом в Лаишевском районе...,103.0,"{""countryId"":138,""undergrounds"":[],""calculated...","{""materialType"":""brick"",""parking"":{},""floorsCo...","{""status"":""individualHousingConstruction"",""are...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,no,NaN,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022-08-01


##### Удалим дубликаты (они есть!)

In [6]:
data1.drop_duplicates(subset=['id'], inplace=True)
data2.drop_duplicates(subset=['id'], inplace=True)
data3.drop_duplicates(subset=['id'], inplace=True)
print(data1.shape, data2.shape, data3.shape)

(1000000, 35) (1000000, 35) (778005, 35)


In [7]:
data1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1000000 entries, 0 to 999999
Data columns (total 35 columns):
 #   Column               Non-Null Count    Dtype  
---  ------               --------------    -----  
 0   id                   1000000 non-null  int64  
 1   floornumber          680725 non-null   float64
 2   category             1000000 non-null  object 
 3   bargainterms         1000000 non-null  object 
 4   description          999994 non-null   object 
 5   totalarea            896852 non-null   float64
 6   geo                  1000000 non-null  object 
 7   building             1000000 non-null  object 
 8   land                 1000000 non-null  object 
 9   hasfurniture         147098 non-null   object 
 10  roomscount           559022 non-null   float64
 11  hasconditioner       75554 non-null    object 
 12  haskitchenfurniture  134796 non-null   object 
 13  hastv                116003 non-null   object 
 14  haswasher            136491 non-null   object 
 15 

In [8]:
data = pd.concat([data1, data2, data3]) # не тестовый вариант
#data = data1.iloc[:10000] # тестовый вариант для ускорения
print('after concating:', data.shape)
data.drop_duplicates(subset=['id'], inplace=True)
print('after drop duplicates:', data.shape)
ID = np.array(data['id']) # сохраним id объявок

after concating: (2778005, 35)
after drop duplicates: (2778005, 35)


Удаляем столбцы, в которых более 90% пропусков

In [9]:
def get_nan_cols(df, nan_percent=0.8):
    """
        функция для определения колонок, в которых много NaN
    """
    threshold = len(df.index) * nan_percent
    return [c for c in df.columns if df[c].isna().sum() >= threshold]

colss = get_nan_cols(data, 0.9)
data = data.drop(colss, axis=1)
data.drop(['ptn_dadd'], axis=1, inplace=True) # неинформативный признак даты, удаляем
#data.tail()

In [10]:
# посмотрим что находится в geo
data['geo'].iloc[0]

'{"countryId":138,"undergrounds":[],"calculatedUndergrounds":[],"coordinates":{"lat":55.718725,"lng":37.272026},"highways":[{"distance":7.0,"id":27,"name":"Рублево-Успенское","isDefault":true},{"distance":15.0,"id":16,"name":"Минское"}],"railways":[{"id":800,"directionIds":[11],"distance":3.0,"time":36,"name":"Барвиха","travelType":"byFoot"},{"id":800,"directionIds":[11],"distance":3.0,"time":4,"name":"Барвиха","travelType":"byCar"},{"id":789,"directionIds":[11],"distance":3.0,"time":39,"name":"Раздоры","travelType":"byFoot"},{"id":789,"directionIds":[11],"distance":4.0,"time":11,"name":"Раздоры","travelType":"byCar"},{"id":807,"directionIds":[11],"distance":4.0,"time":53,"name":"Ильинское","travelType":"byFoot"},{"id":807,"directionIds":[11],"distance":5.0,"time":8,"name":"Ильинское","travelType":"byCar"}],"userInput":"Московская область, Одинцовский городской округ, Рождественно деревня, Барвиха Клаб территория","address":[{"name":"Московская","id":4593,"locationTypeId":2,"fullName":

### Преобразуем признак geo

In [11]:
data['countryId'] = data['geo'].progress_apply(lambda x: json.loads(x)['countryId'])
data['lat'] = data['geo'].progress_apply(lambda x: json.loads(x)['coordinates']['lat'])
data['lng'] = data['geo'].progress_apply(lambda x: json.loads(x)['coordinates']['lng'])
data.drop(['geo'], axis=1, inplace=True)
data.head()

100%|███████████████████████████████| 2778005/2778005 [24:09<00:00, 1916.92it/s]


,id,floornumber,category,bargainterms,description,totalarea,building,land,hasfurniture,roomscount,haskitchenfurniture,hastv,haswasher,hasbathtub,bedroomscount,repairtype,petsallowed,hasfridge,wclocationtype,kitchenarea,isapartments,loggiascount,balconiescount,allroomsarea,livingarea,flattype,combinedwcscount,countryId,lat,lng
0,191401467,NaN,townhouseSale,"{""price"":5.5E7,""currency"":""rur"",""priceType"":""a...",Все предложения в Барвихе! ОКП Барвиха Клаб. В...,400.00,"{""materialType"":""monolith"",""parking"":{},""floor...","{""status"":""individualHousingConstruction"",""are...",NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,indoors,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,138,55.718725,37.272026
1,212158376,NaN,commercialLandRent,"{""clientFee"":0,""leaseTermType"":""longTerm"",""pri...",Сдается от собственника в аренду площадка 560 ...,NaN,"{""parking"":{},""cranageTypes"":[],""extinguishing...","{""status"":""settlements"",""possibleToChangeStatu...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,138,61.809605,34.296402
2,227815832,NaN,landSale,"{""price"":240000.0,""currency"":""rur"",""priceType""...",Продам участок в жилом районе с. ХОМУТОВО,NaN,"{""parking"":{},""cranageTypes"":[],""extinguishing...","{""status"":""individualHousingConstruction"",""are...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,138,52.485696,104.341456
3,238017518,1.0,flatRent,"{""clientFee"":0,""leaseTermType"":""longTerm"",""pri...","Привет, наконец ты попал сюда!\n\nУютная студи...",27.00,"{""materialType"":""brick"",""parking"":{},""floorsCo...",{},True,NaN,True,NaN,True,True,NaN,euro,False,True,NaN,NaN,False,NaN,1.0,NaN,NaN,studio,NaN,138,45.021564,38.925492
4,241597454,20.0,flatSale,"{""price"":1.6E7,""currency"":""rur"",""priceType"":""a...",Видовой 20 этаж. Корпус Д. Срочная перепродажа...,60.65,"{""name"":""ЖК «CASABLANKA (Касабланка)»"",""materi...",{},NaN,2.0,NaN,NaN,NaN,NaN,NaN,no,NaN,NaN,NaN,22.0,False,1.0,1.0,18+20,NaN,rooms,NaN,138,43.485547,39.893921


### Преобразуем признаки-словари bargainterms, building, land

Составим новые колонки из ключей словарей исходных колонок

In [12]:
data['bargainterms'] = data['bargainterms'].progress_apply(json.loads)
keys_bargainterms = data['bargainterms'].apply(lambda x: x.keys()).explode().unique()
bargainterms_df = pd.DataFrame(data['bargainterms'].values.tolist())

data['building'] = data['building'].progress_apply(json.loads)
keys_building = data['building'].apply(lambda x: x.keys()).explode().unique()
building_df = pd.DataFrame(data['building'].values.tolist())

data['land'] = data['land'].progress_apply(json.loads)
keys_land = data['land'].apply(lambda x: x.keys()).explode().unique()
land_df = pd.DataFrame(data['land'].values.tolist())

land_df.rename(columns={'type': 'land_type'}, inplace=True)
#data = data.join(bargainterms_df).join(building_df).join(land_df)
#print(data.shape)
#data.head()

100%|█████████████████████████████| 2778005/2778005 [00:10<00:00, 273715.09it/s]


In [13]:
# удаление строковых колонок
def del_str_cols(df):
    columns = df.columns
    for col in columns:
        if type(df[col].iloc[1]) == str:
            df.drop(col, inplace=True, axis=1)
    return df

print(data.shape)
print(bargainterms_df.shape)
print(building_df.shape)
print(land_df.shape)

data = del_str_cols(data)
bargainterms_df = del_str_cols(bargainterms_df)
building_df = del_str_cols(building_df)
land_df = del_str_cols(land_df)

print(data.shape)
print(bargainterms_df.shape)
print(building_df.shape)
print(land_df.shape)

(2778005, 30)
(2778005, 28)
(2778005, 33)
(2778005, 5)
(2778005, 28)
(2778005, 22)
(2778005, 33)
(2778005, 3)


In [14]:
# почистим колонки с большим кол-вом пропусков и удалим ненужные словарные
data.drop(['bargainterms', 'building', 'land'], axis=1, inplace=True)
colss = get_nan_cols(data, 0.9)
data.drop(colss, axis=1, inplace=True)

colss = get_nan_cols(bargainterms_df, 0.9)
bargainterms_df.drop(colss, axis=1, inplace=True)

colss = get_nan_cols(building_df, 0.9)
building_df.drop(colss, axis=1, inplace=True)

colss = get_nan_cols(land_df, 0.9)
land_df.drop(colss, axis=1, inplace=True)

print(data.shape)
print(bargainterms_df.shape)
print(building_df.shape)
print(land_df.shape)

(2778005, 25)
(2778005, 9)
(2778005, 17)
(2778005, 1)


In [50]:
a = np.array([[1, 2, 3], [4, 5, 6]])
np.max(a, axis=0)

array([4, 5, 6])

In [15]:
display(data.head())
display(bargainterms_df.head())
display(building_df.head())
land_df.head()

,id,floornumber,totalarea,hasfurniture,roomscount,haskitchenfurniture,hastv,haswasher,hasbathtub,bedroomscount,repairtype,petsallowed,hasfridge,wclocationtype,kitchenarea,isapartments,loggiascount,balconiescount,allroomsarea,livingarea,flattype,combinedwcscount,countryId,lat,lng
0,191401467,NaN,400.00,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,indoors,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,138,55.718725,37.272026
1,212158376,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,138,61.809605,34.296402
2,227815832,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,138,52.485696,104.341456
3,238017518,1.0,27.00,True,NaN,True,NaN,True,True,NaN,euro,False,True,NaN,NaN,False,NaN,1.0,NaN,NaN,studio,NaN,138,45.021564,38.925492
4,241597454,20.0,60.65,NaN,2.0,NaN,NaN,NaN,NaN,NaN,no,NaN,NaN,NaN,22.0,False,1.0,1.0,18+20,NaN,rooms,NaN,138,43.485547,39.893921


,price,includedOptions,mortgageAllowed,clientFee,agentFee,prepayMonths,deposit,utilitiesTerms,saleType
0,55000000.0,[],True,NaN,NaN,NaN,NaN,NaN,NaN
1,6000.0,[],NaN,0.0,0.0,1.0,NaN,NaN,NaN
2,240000.0,[],NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,12990.0,[],NaN,0.0,0.0,1.0,8000.0,"{'includedInPrice': False, 'price': 1.0, 'flow...",NaN
4,16000000.0,[],True,NaN,NaN,NaN,NaN,NaN,free


,materialType,parking,floorsCount,heatingType,cranageTypes,extinguishingSystemTypes,liftTypes,infrastructure,totalArea,openingHours,deadline,name,hasGarbageChute,passengerLiftsCount,buildYear,cargoLiftsCount,ceilingHeight
0,monolith,{},4.0,centralGas,[],[],[],{},400.0,{},{},NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,{},NaN,NaN,[],[],[],{},NaN,{},{},NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,{},NaN,NaN,[],[],[],{},NaN,{},{},NaN,NaN,NaN,NaN,NaN,NaN
3,brick,{},3.0,NaN,[],[],[],{},27.0,{},{},NaN,NaN,NaN,NaN,NaN,NaN
4,monolith,{'type': 'multilevel'},20.0,NaN,[],[],[],{},60.7,{},{},ЖК «CASABLANKA (Касабланка)»,True,3.0,2014.0,NaN,NaN


,area
0,2.0
1,5.0
2,10.0
3,NaN
4,NaN


In [16]:
data.drop(['allroomsarea', 'repairtype', 'flattype'], axis=1, inplace=True)
bargainterms_df.drop(['includedOptions'], axis=1, inplace=True)
building_df.drop(['cranageTypes', 'extinguishingSystemTypes', 'liftTypes'], axis=1, inplace=True)

In [17]:
data = data.join(bargainterms_df).join(building_df).join(land_df)

In [18]:
data.shape

(2778005, 45)

##### Оставшиеся колонки типа list: 
includedOptions, cranageTypes, extinguishingSystemTypes, liftTypes	

##### Оставшиеся колонки типа dict: 
utilitiesTerms, parking, infrastructure, openingHours, deadline, agentBonus

##### Тип пока непонятен для колонок: 
bedroomscount, wclocationtype, allroomsarea, name, heatingType, areaUnitType, area, status

In [19]:
# посмотрим, сколько пустых словарей/списков
data['utilitiesTerms'] = data['utilitiesTerms'].progress_apply(lambda y: np.nan if y is None else y)
data['parking'] = data['parking'].progress_apply(lambda y: np.nan if len(y)==0 else y)
data['infrastructure'] = data['infrastructure'].progress_apply(lambda y: np.nan if len(y)==0 else y)
data['openingHours'] = data['openingHours'].progress_apply(lambda y: np.nan if len(y)==0 else y)
data['deadline'] = data['deadline'].progress_apply(lambda y: np.nan if len(y)==0 else y)
#data['agentBonus'] = data['agentBonus'].progress_apply(lambda y: np.nan if y is None else y)

#data['includedOptions'] = data['includedOptions'].progress_apply(lambda y: np.nan if y is None else y)
#data['cranageTypes'] = data['cranageTypes'].progress_apply(lambda y: np.nan if y is None else y)
#data['extinguishingSystemTypes'] = data['extinguishingSystemTypes'].progress_apply(lambda y: np.nan if y is None else y)
#data['liftTypes'] = data['liftTypes'].progress_apply(lambda y: np.nan if y is None else y)
data.head()

100%|█████████████████████████████| 2778005/2778005 [00:03<00:00, 797378.81it/s]


,id,floornumber,totalarea,hasfurniture,roomscount,haskitchenfurniture,hastv,haswasher,hasbathtub,bedroomscount,petsallowed,hasfridge,wclocationtype,kitchenarea,isapartments,loggiascount,balconiescount,livingarea,combinedwcscount,countryId,lat,lng,price,mortgageAllowed,clientFee,agentFee,prepayMonths,deposit,utilitiesTerms,saleType,materialType,parking,floorsCount,heatingType,infrastructure,totalArea,openingHours,deadline,name,hasGarbageChute,passengerLiftsCount,buildYear,cargoLiftsCount,ceilingHeight,area
0,191401467,NaN,400.0,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,indoors,NaN,NaN,NaN,NaN,NaN,NaN,138,55.718725,37.272026,55000000.0,True,NaN,NaN,NaN,NaN,NaN,NaN,monolith,NaN,4.0,centralGas,NaN,400.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0
0,275557007,13.0,24.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,NaN,NaN,NaN,1.0,138,45.073681,38.952568,55000000.0,True,NaN,NaN,NaN,NaN,NaN,NaN,monolith,NaN,4.0,centralGas,NaN,400.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0
0,274482535,NaN,40.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,138,55.644999,37.860899,55000000.0,True,NaN,NaN,NaN,NaN,NaN,NaN,monolith,NaN,4.0,centralGas,NaN,400.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0
1,212158376,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,138,61.809605,34.296402,6000.0,NaN,0.0,0.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0
1,275619810,4.0,74.5,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,1.0,NaN,0.0,138,61.234358,73.402025,6000.0,NaN,0.0,0.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0


In [20]:
# традиционно почистим от колонок, в которых много пропусков
print(data.shape)
colss = get_nan_cols(data, 0.9)
data = data.drop(colss, axis=1)
print(data.shape)

(2778005, 45)
(2778005, 42)


##### Странные значения в колонках descriptions, allroomsarea, includedOptions, agentBonus, name
##### И в числовых cranageTypes, extinguishingSystemTypes, liftTypes
удалим их

In [22]:
data.drop(['description', 'allroomsarea', 'includedOptions'], axis=1, inplace=True) # 'name', 'agentBonus' уже пропали
data.drop('liftTypes', axis=1, inplace=True) # 'cranageTypes' 'extinguishSystemTypes' тоже

KeyError: "['description' 'allroomsarea' 'includedOptions'] not found in axis"

In [21]:
# нужно переименовать ключ price в utilitiesTerms и type в parking
def rename_price(arg):
    if arg is np.nan:
        return arg
    try:
        arg['comm_price'] = arg['price']
        arg.pop('price')
        return arg
    except:
        pass
data['utilitiesTerms'].apply(rename_price)

def rename_type(arg):
    if arg is np.nan:
        return arg
    try:
        arg['parking_type'] = arg['type']
        arg.pop('type')
        return arg
    except:
        pass
data['parking'].apply(rename_type)

0         NaN
0         NaN
0         NaN
1         NaN
1         NaN
         ... 
999997    NaN
999998    NaN
999998    NaN
999999    NaN
999999    NaN
Name: parking, Length: 2778005, dtype: object

In [ ]:
##### for i in tqdm(list(...)):

In [81]:
#print(data['utilitiesTerms'].iloc[20])
#print(data['parking'].iloc[20])

In [23]:
data.head()

,id,floornumber,totalarea,hasfurniture,roomscount,haskitchenfurniture,hastv,haswasher,hasbathtub,bedroomscount,petsallowed,hasfridge,wclocationtype,kitchenarea,isapartments,loggiascount,balconiescount,livingarea,combinedwcscount,countryId,lat,lng,price,mortgageAllowed,clientFee,agentFee,prepayMonths,deposit,utilitiesTerms,saleType,materialType,parking,floorsCount,heatingType,totalArea,deadline,hasGarbageChute,passengerLiftsCount,buildYear,cargoLiftsCount,ceilingHeight,area
0,191401467,NaN,400.0,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,indoors,NaN,NaN,NaN,NaN,NaN,NaN,138,55.718725,37.272026,55000000.0,True,NaN,NaN,NaN,NaN,NaN,NaN,monolith,NaN,4.0,centralGas,400.0,NaN,NaN,NaN,NaN,NaN,NaN,2.0
0,275557007,13.0,24.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,NaN,NaN,NaN,1.0,138,45.073681,38.952568,55000000.0,True,NaN,NaN,NaN,NaN,NaN,NaN,monolith,NaN,4.0,centralGas,400.0,NaN,NaN,NaN,NaN,NaN,NaN,2.0
0,274482535,NaN,40.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,138,55.644999,37.860899,55000000.0,True,NaN,NaN,NaN,NaN,NaN,NaN,monolith,NaN,4.0,centralGas,400.0,NaN,NaN,NaN,NaN,NaN,NaN,2.0
1,212158376,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,138,61.809605,34.296402,6000.0,NaN,0.0,0.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0
1,275619810,4.0,74.5,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,1.0,NaN,0.0,138,61.234358,73.402025,6000.0,NaN,0.0,0.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0


##### ещё словарные признаки - utilitiesTerms, parking - распакуем их

In [24]:
data[['parking', 'utilitiesTerms']] = data[['parking', 'utilitiesTerms']].applymap(lambda x: \
                                                                                   {} if pd.isnull(x) else x)
parking_df = pd.DataFrame(data['parking'].values.tolist())
utilitiesTerms_df = pd.DataFrame(data['utilitiesTerms'].dropna().values.tolist())

parking_df.rename(columns={'currency': 'curr'}, inplace=True)
data = data.join(parking_df).join(utilitiesTerms_df)

In [25]:
data = data.drop(['parking', 'utilitiesTerms'], axis=1)

In [26]:
colss = get_nan_cols(data, 0.9)
data = data.drop(colss, axis=1)
data.head()

,id,floornumber,totalarea,hasfurniture,roomscount,haskitchenfurniture,hastv,haswasher,hasbathtub,bedroomscount,petsallowed,hasfridge,wclocationtype,kitchenarea,isapartments,loggiascount,balconiescount,livingarea,combinedwcscount,countryId,lat,lng,price,mortgageAllowed,clientFee,agentFee,prepayMonths,deposit,saleType,materialType,floorsCount,heatingType,totalArea,deadline,hasGarbageChute,passengerLiftsCount,buildYear,cargoLiftsCount,ceilingHeight,area,parking_type
0,191401467,NaN,400.0,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,indoors,NaN,NaN,NaN,NaN,NaN,NaN,138,55.718725,37.272026,55000000.0,True,NaN,NaN,NaN,NaN,NaN,monolith,4.0,centralGas,400.0,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN
0,275557007,13.0,24.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,NaN,NaN,NaN,1.0,138,45.073681,38.952568,55000000.0,True,NaN,NaN,NaN,NaN,NaN,monolith,4.0,centralGas,400.0,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN
0,274482535,NaN,40.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,138,55.644999,37.860899,55000000.0,True,NaN,NaN,NaN,NaN,NaN,monolith,4.0,centralGas,400.0,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN
1,212158376,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,138,61.809605,34.296402,6000.0,NaN,0.0,0.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN
1,275619810,4.0,74.5,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,1.0,NaN,0.0,138,61.234358,73.402025,6000.0,NaN,0.0,0.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN


### Преобразование категориальных признаков

In [27]:
# выделим категориальные и числовые признаки
types = data.dtypes
numerical_cols = []
cat_cols = []
for i in tqdm(range(len(types))):
    if types[i] != 'object':
        numerical_cols.append(data.columns[i])
    else:
        cat_cols.append(data.columns[i])
#print(types)
#print(numerical_cols)
#print(cat_cols)

100%|███████████████████████████████████████| 41/41 [00:00<00:00, 181399.22it/s]


In [28]:
data[cat_cols].head()

,hasfurniture,haskitchenfurniture,hastv,haswasher,hasbathtub,petsallowed,hasfridge,wclocationtype,isapartments,mortgageAllowed,saleType,materialType,heatingType,deadline,hasGarbageChute,parking_type
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,indoors,NaN,True,NaN,monolith,centralGas,NaN,NaN,NaN
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,True,NaN,monolith,centralGas,NaN,NaN,NaN
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,NaN,monolith,centralGas,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [29]:
data[numerical_cols].shape

(2778005, 25)

In [30]:
data.drop(cat_cols, inplace=True, axis=1)

In [31]:
# обработка пропусков
# для числовых признаков
def fillna_num(data_col):
    return data_col.fillna(data_col.mean())

# для категориальных
#def fillna_cat(data_col):
#    return data_col.fillna('-1-')

for col in tqdm(numerical_cols):
    data[col] = fillna_num(data[col])
    
#for col in tqdm(cat_cols):
#    data[col] = fillna_cat(data[col])

data.head()

100%|███████████████████████████████████████████| 25/25 [00:00<00:00, 30.86it/s]


,id,floornumber,totalarea,roomscount,bedroomscount,kitchenarea,loggiascount,balconiescount,livingarea,combinedwcscount,countryId,lat,lng,price,clientFee,agentFee,prepayMonths,deposit,floorsCount,totalArea,passengerLiftsCount,buildYear,cargoLiftsCount,ceilingHeight,area
0,191401467,6.158313,400.000000,1.966945,5.000000,10.839416,0.528029,0.646426,30.759469,0.73138,138,55.718725,37.272026,55000000.0,27.896778,24.195109,1.042697,600502.243938,4.000000,400.000000,1.042613,1994.235776,0.740304,3.037771,2.0
0,275557007,13.000000,24.000000,1.966945,3.242101,10.839416,0.528029,0.646426,30.759469,1.00000,138,45.073681,38.952568,55000000.0,27.896778,24.195109,1.042697,600502.243938,4.000000,400.000000,1.042613,1994.235776,0.740304,3.037771,2.0
0,274482535,6.158313,40.000000,1.966945,3.242101,10.839416,0.528029,0.646426,30.759469,0.73138,138,55.644999,37.860899,55000000.0,27.896778,24.195109,1.042697,600502.243938,4.000000,400.000000,1.042613,1994.235776,0.740304,3.037771,2.0
1,212158376,6.158313,197.994591,1.966945,3.242101,10.839416,0.528029,0.646426,30.759469,0.73138,138,61.809605,34.296402,6000.0,0.000000,0.000000,1.000000,600502.243938,9.698875,483.570875,1.042613,1994.235776,0.740304,3.037771,5.0
1,275619810,4.000000,74.500000,3.000000,3.242101,10.839416,0.000000,1.000000,30.759469,0.00000,138,61.234358,73.402025,6000.0,0.000000,0.000000,1.000000,600502.243938,9.698875,483.570875,1.042613,1994.235776,0.740304,3.037771,5.0


In [32]:
# кодирование категориальных признаков
#data_cat = data[cat_cols]
#enc = OneHotEncoder(handle_unknown='ignore')
#cat_enc = enc.fit_transform(data_cat.astype(str)).toarray()
#display(cat_enc)
#print(cat_enc.shape)

In [33]:
# обработка description
#vectorizer = TfidfVectorizer(max_features=5000)
#descr_vect = vectorizer.fit_transform(data['description']).toarray()
#print(descr_vect.shape)
#display(descr_vect)

In [34]:
#vectorizer.vocabulary_

In [35]:
# отмасштабируем числовые признаки, кроме id
num_cols = data[numerical_cols]
num_cols = num_cols.drop(['id'], axis=1)
scaler = MinMaxScaler()
num_cols = scaler.fit_transform(num_cols)
print(np.max(num_cols), np.min(num_cols))

1.0000000000000002 0.0


In [36]:
#embeddings = np.hstack((num_cols, cat_enc))
embeddings = np.hstack((ID.reshape((len(ID), 1)), num_cols))
print(embeddings.shape)
embeddings

(2778005, 25)


array([[1.91401467e+08, 7.04485578e-02, 1.73478268e-05, ...,
        1.89821606e-02, 7.59442807e-04, 4.76559242e-08],
       [2.12158376e+08, 1.23076923e-01, 1.00000004e-06, ...,
        1.89821606e-02, 7.59442807e-04, 4.76559242e-08],
       [2.27815832e+08, 7.04485578e-02, 1.69565225e-06, ...,
        1.89821606e-02, 7.59442807e-04, 4.76559242e-08],
       ...,
       [2.75672461e+08, 3.07692308e-02, 3.50000015e-05, ...,
        1.89821606e-02, 7.37500000e-04, 1.67815467e-05],
       [2.75950340e+08, 6.92307692e-02, 2.34782619e-06, ...,
        1.89821606e-02, 6.87500000e-04, 1.67815467e-05],
       [2.76233360e+08, 3.84615385e-02, 1.30434788e-06, ...,
        1.89821606e-02, 6.87500000e-04, 1.67815467e-05]])

In [37]:
ID # сохранённый список id

array([191401467, 212158376, 227815832, ..., 275672461, 275950340,
       276233360])

In [38]:
type(embeddings)

numpy.ndarray

### Сохранение в разных форматах

In [39]:
# save as .txt
#np.savetxt('embeddings_test10000.txt', embeddings, delimiter = ' ')

In [40]:
# save as sparse matrix
#sparse_embeddings = csc_matrix(embeddings)
#save_npz('embeddings_test10000.npz', sparse_embeddings)

In [41]:
# save in binary format
np.save('embeddings_num_only.npy', embeddings)  

In [42]:
# save ID
np.save('embeddings_ID.npy', embeddings[:, 0])  

In [44]:
# make dict
#embeddings = np.hstack((num_cols, cat_enc))
print(ID.shape, type(ID))
print(embeddings.shape, type(embeddings))
embeddingsID = {}
for i in range(len(ID)):
    embeddingsID[ID[i]] = embeddings[i]
#print(embeddingsID)

(2778005,) <class 'numpy.ndarray'>
(2778005, 25) <class 'numpy.ndarray'>
